<a href="https://colab.research.google.com/github/hxk271/SocDataSci/blob/main/archive/W14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘의 파이썬

아래 연습 자료를 다운로드받자(로컬에서 테스트한다면 모든 파일을 여러분의 로컬 컴퓨터에도 다운로드 받아두어야 한다).

In [ ]:
import gdown
links = ['https://drive.google.com/uc?id=1oGH3n0gIAfkhqTT06xYy2BoH5LWpS070',          #ds_salary
         'https://drive.google.com/uc?id=1oEULmOXaUroX0MMn43mhZF-mqi3Crcwr',          #medical_cost
         'https://drive.google.com/uc?id=1rsYj2AZ5PEuY9GMm0jIqjLCFecIbfeHg']          #CO2_emissions

for link in links:
    gdown.download(link)

## 1. `with`를 사용한 블록

> 학기 초 우리가 배운 바에 따르면, 파일을 불러와 화면에 출력할 때 아래처럼 해야 했다.

In [ ]:
tempfile = open('medical_cost.csv', 'r', encoding = 'cp949')
data = tempfile.read()
print(data)
tempfile.close()

> 그런데 `with`를 사용한다면, 이 파일이 필요한 특정 구간에서만 자원을 할당하여 잠깐 쓰고 버릴 수 있게 된다. 이처럼 `with` 블록은 리소스의 생명주기를 관리하기 때문에 (특히 큰 프로그램을 만들 때) 제법 유용하다.

In [ ]:
with open('medical_cost.csv', 'r', encoding = 'cp949') as tempfile:
    data = tempfile.read()
    print(data)

> `with` 블록 안에서 변수 할당을 한 경우 그건 살아있지만, `tempfile`로 열었던 파일은 저절로 닫힌다. 따라서 `with` 블록이 끝난 뒤 다시 열려고 시도하면 에러가 부딪친다.

In [ ]:
with open('medical_cost.csv', 'r', encoding = 'cp949') as tempfile:
    data = tempfile.read()
    print(data)

print(data)                 #여기선 문제없지만,
data = tempfile.read()      #여기선 문제가 있다.

# Week 14 (데이터 대시보드)

오늘은 마지막 수업이다. 이제 최종적으로 <b>데이터 대시보드(data databoard)</b>를 직접 구축한다. (1) 데이터를 pandas에서 필터링하고, (2) plotly나 seaborn을 사용하여 시각화 하고, (3) streamlit으로 대시보드 디자인을 하여 결과물을 생산하는 순서를 따른다. 지금까지의 모든 과정은 사실 이미 배웠다. 그러므로 오늘 내용은 복습에 가깝다.
<br><br>
수업을 위해 몇 가지 준비가 필요하다. 지난 주에 배웠듯, 만일 여러분이 npm을 사용하여 streamlit 웹 대시보드를 구현한다면 `!pip install streamlit` 과 `!npm install localtunnel` 등으로 환경 설정을 해야 한다. 실행할 때는 `!streamlit run app.py & >logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com`로 실행한다. 이것보다는 로컬에서 테스트해보는 게 편할 수 있고, 여러분이 선호하는 쪽을 선택하자.

In [ ]:
import gdown
links = ['https://drive.google.com/uc?id=1oGH3n0gIAfkhqTT06xYy2BoH5LWpS070',          #ds_salary
         'https://drive.google.com/uc?id=1oEULmOXaUroX0MMn43mhZF-mqi3Crcwr',          #medical_cost
         'https://drive.google.com/uc?id=1rsYj2AZ5PEuY9GMm0jIqjLCFecIbfeHg']          #CO2_emissions

for link in links:
    gdown.download(link)

## 2. 이용자 주도 분석

> 이제부터 인터렉티브하게 이용자가 원하는 방식으로 plotly 시각화 자료를 만들어 나갈 수 있도록 해보자. 먼저 tips 데이터셋의 $x$, $y$, 색조를 지정하고 콜랩 환경에서 직접 출력해보자.

In [ ]:
import seaborn as sns
import plotly.express as px

df = sns.load_dataset('tips')
df

# 옵션 지정 (바꿔보자)
x_option = 'day'
y_option = 'total_bill'

# 시각화
fig = px.box(data_frame=df,
             x=x_option, y=y_option, width=500)
fig.show()

> 데이터 대시보드의 강점은 사용자가 직접 원하는 분석을 주도할 수 있다는 점이다(Why?). 사용자의 선호를 받아들이기 위해 위젯을 활용해야 한다. `st.selectbox`를 사용하여 변수를 고를 수 있도록 해보자. 아래 코드에서 `x_option`과 `y_option`에 무엇이 들어갈 수 있는지 유심히 생각해 보아야 한다.
>
> 또한 지금부터는 콜랩 환경이 아니라 웹 대시보드를 퍼블리시해야 한다. streamlit을 위한 소스 코드를 작성하고 서버에 퍼블리시하자.

In [ ]:
%%writefile app.py

import streamlit as st
import seaborn as sns
import plotly.express as px

#tip data set
df = sns.load_dataset('tips')

#selectbox
x_option = st.selectbox(label='Select X-axis',
                        index=None,
                        options=['day','size'])
y_option = st.selectbox(label='Select Y-axis',
                        index=None,
                        options=['total_bill','tip'])

#visualization
if (x_option != None) & (y_option != None):

    fig = px.box(data_frame=df,
                 x=x_option, y=y_option,
                 width=500)
    st.plotly_chart(fig)

> **연습문제 2-1**. 위 코드는 $x$와 $y$ 축(axis)으로 선택할 수 있는 변수 옵션을 개발자가 미리 제공해야만 했다. 이번엔 데이터에서 사용가능한 "모든" 변수가 바로 출력되어 사용자가 자유롭게 고를 수 있도록 수정하시오.

In [ ]:
%%writefile app.py

import streamlit as st
import seaborn as sns
import plotly.express as px

#tip data set
df = sns.load_dataset('tips')

#selectbox
x_option = st.selectbox(label='Select X-axis',
                        index=None,
                        options=df.columns)
y_option = st.selectbox(label='Select Y-axis',
                        index=None,
                        options=df.columns)

#visualization
if (x_option != None) & (y_option != None):

    fig = px.box(data_frame=df,
                 x=x_option, y=y_option,
                 width=500)
    st.plotly_chart(fig)

> 앞서 만든 코드를 좀 더 수정해보자. `px.box()`의 `color` 옵션을 제공하면 특정 변수별로 그래프를 대조한다. 이것까지도 선택할 수 있도록 개선해보자.

In [ ]:
%%writefile app.py

import streamlit as st
import seaborn as sns
import plotly.express as px

df = sns.load_dataset('tips')

# 옵션을 입력받기
x_option = st.selectbox(label='Select X-axis',
                        index=None,
                        options=['day','size'])
y_option = st.selectbox(label='Select Y-axis',
                        index=None,
                        options=['total_bill','tip'])
hue_option = st.selectbox(label='Select Hue',
                          index=None,
                          options=['smoker','sex'])

# 시각화
if (x_option != None) & (y_option != None) & (hue_option != None):

    fig = px.box(data_frame=df,
                 x=x_option, y=y_option, color=hue_option,
                 width=500)
    st.plotly_chart(fig)

> **연습문제 2-2**. 위 연습문제에서 hue를 고를 수 있도록 수정하시오. 사용자가 hue를 고른 경우에 한하여 해당 변수 그룹 별로 색조를 구분하도록 하시오.

In [ ]:
%%writefile app.py

import streamlit as st
import seaborn as sns
import plotly.express as px

df = sns.load_dataset('tips')

# 옵션을 입력받기
x_option = st.selectbox(label='Select X-axis',
                        index=None,
                        options=df.columns)
y_option = st.selectbox(label='Select Y-axis',
                        index=None,
                        options=df.columns)
hue_option = st.selectbox(label='Select Hue',
                          index=None,
                          options=df.columns)

# 시각화
if (x_option != None) & (y_option != None):

    # hue를 골랐을 경우
    if hue_option != None:
        fig = px.box(data_frame=df,
                     x=x_option, y=y_option, color=hue_option,
                     width=500)

    # hue를 고르지 않았을 경우
    else:
        fig = px.box(data_frame=df,
                     x=x_option, y=y_option, width=500)

    st.plotly_chart(fig)

## 3. 이미지 출력

> 파이썬에서 간단한 이미지 조작과 출력을 한다면 Pillow 라이브러리(`PIL`)가 매우 편리하다. 웹에서 적당히 이미지 파일을 하나 다운로드받아 이를 여러분의 로컬 컴퓨터(=서버)에 `image.jpg`라는 이름으로 저장하고 직접 출력해보자(폴더 위치에도 주의하자).

In [ ]:
%%writefile app.py

import streamlit as st

#Python Imaging Library
from PIL import Image

#PIL 라이브러를 활용하여 서버 내부의 파일 접근(원칙적으로는 옳지 않은 접근 방식이다).
img = Image.open('C:/Users/user/Downloads/image.jpg')

#streamlit에서 대시보드로 출력
st.image(image = img,
         width=800,
         caption='멋진 이미지')

> **연습문제 3-1**. 사용자로부터 jpg, png, gif 세 포멧의 이미지를 업로드 받아 이를 출력하는 이미지 뷰어(image viewer) 유틸리티 어플리케이션을 만드시오.

In [ ]:
%%writefile app.py

import streamlit as st
from PIL import Image

# file uploader
img = st.file_uploader(label = '업로드할 JPG 파일을 선택하시오',
                       type=['jpg','png','gif'],
                       accept_multiple_files=False)

if img is not None:
    pil_img = Image.open(img)
    st.image(image = pil_img,
             width=800,
             caption='당신이 보고 있는 파일은 ' + img.name + ' 입니다.')

## 4. 레이아웃

> 화면을 꾸미는 방식을 일컫어 <b>레이아웃(layout)</b>이라고 부른다. 가장 먼저 사용자가 원하는대로 켜고 끌 수 있는 <b>사이드바(sidebar)</b>를 만들어보자. 그 안에 무엇을 집어넣을 것인가는 여러분의 취향에 달렸다. 아까 배운대로 `with` 블록 안에 `st.sidebar`를 사용하자.

In [ ]:
%%writefile app.py

import streamlit as st

#sidebar
with st.sidebar:

    st.title('사이드바')

    # selectbox
    available_options=['첫번째 이미지', '두번째 이미지']
    option = st.selectbox(label='무엇을 먼저 보고 싶니?',
                          index=None,           #아무것도 안골랐을때 할당되는 값은 None
                          options=available_options)

#본문
st.title('메인 페이지')

> 먼저 아래 코드의 결과를 확인하기에 앞서, 웹에서 이미지 파일을 또하나 다운로드받아 두자. 이름은 `image2.jpg`라고 하자.


In [ ]:
%%writefile app.py

import streamlit as st
from PIL import Image

#sidebar
with st.sidebar:

    st.title('사이드바')

    # selectbox
    available_options=['첫번째 이미지', '두번째 이미지']
    option = st.selectbox(label='무엇을 먼저 보고 싶니?',
                          index=None,           #아무것도 안골랐을때 할당되는 값은 None
                          options=available_options)

#본문
st.title('너가 고른 이미지')

st.header('이것이 헤더다!')

#좋지 않은 방식
imgfile1 = 'C:/Users/user/Downloads/image.jpg'
img1 = Image.open(imgfile1)
imgfile2 = 'C:/Users/user/Downloads/image2.jpg'
img2 = Image.open(imgfile2)

if option == available_options[0]:
    st.image(img1, width=900, caption='첫번째')
    st.image(img2, width=900, caption='두번째')

elif option == available_options[1]:
    st.image(img2, width=900, caption='첫번째')
    st.image(img1, width=900, caption='두번째')

else:
    st.markdown("아직 아무것도 안골랐지")

> 여러 그림이 위아래로 배치되는 방식도 나쁘지 않다. 그런데 기분에 따라 위아래보다는 좌우로 배치하고 싶을수도 있다. 이때 <b>컬럼(columns)</b>을 사용해야 한다. 컬럼은 여러 개 만들 수도 있고, 설정하기에 따라 간격을 조정할 수도 있다. 참고로 아래 코드는 자명함에 역점을 두고 있기 때문에 별로 파이썬답지(pythonic) 않다.

In [ ]:
%%writefile app.py

import streamlit as st
from PIL import Image

#페이지 설정
st.set_page_config(page_title="붕붕이", layout="wide")

#sidebar
with st.sidebar:

    st.title('사이드바')

    # selectbox
    available_options=['첫번째 이미지', '두번째 이미지']
    option = st.selectbox(label='무엇을 먼저 보고 싶니?',
                          index=None,           #아무것도 안골랐을때 할당되는 값은 None
                          options=available_options)

#본문
st.title('너가 고른 이미지')

st.header('이것이 헤더다!')

#좋지 않은 방식
imgfile1 = 'C:/Users/user/Downloads/image.jpg'
img1 = Image.open(imgfile1)
imgfile2 = 'C:/Users/user/Downloads/image2.jpg'
img2 = Image.open(imgfile2)

#컬럼
col1, col2 = st.columns(2)        #st.columns([2, 1])라면 왼쪽이 두 배!

if option == available_options[0]:

    with col1:
        st.image(img1, width=900, caption='첫번째')

    with col2:
        st.image(img2, width=900, caption='두번째')

elif option == available_options[1]:
    with col1:
        st.image(img2, width=900, caption='첫번째')

    with col2:
        st.image(img1, width=900, caption='두번째')

else:
    st.markdown("아직 아무것도 안골랐지")


> 다음으로 <b>탭(tab)</b>을 살펴보기로 한다. 우선 그에 앞서 아래 두 가지 내용을 보여줄 생각인데, 우선 결과를 확인해보자!

In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('medical_cost.csv')
df
df2 = df[df['region'] == "northwest"]
df2

#첫번째 탭
print(df2.head(5))

#두번째 탭
fig = px.scatter(data_frame=df2,
                 x='bmi', y='charges', trendline='ols',
                 width=700)
fig.show()

> 탭 방식은 마치 엑셀처럼 특정 탭으로 넘겨 원하는 종류의 데이터만 볼 수 있게 된다. 주어진 페이지를 절약하기 위해서 사용한다.

In [ ]:
%%writefile app.py

import streamlit as st
from PIL import Image
import pandas as pd
import plotly.express as px

#페이지 설정
st.set_page_config(page_title="붕붕이", layout="wide")

#sidebar
with st.sidebar:

    st.title('사이드바')

    # selectbox
    available_options=['첫번째 이미지', '두번째 이미지']
    option = st.selectbox(label='무엇을 먼저 보고 싶니?',
                          index=None,           #아무것도 안골랐을때 할당되는 값은 None
                          options=available_options)

#본문
st.title('너가 고른 이미지')

st.header('이것이 헤더다!')

#좋지 않은 방식
imgfile1 = 'C:/Users/user/Downloads/image.jpg'
img1 = Image.open(imgfile1)
imgfile2 = 'C:/Users/user/Downloads/image2.jpg'
img2 = Image.open(imgfile2)

#1. 컬럼
col1, col2 = st.columns(2)        #st.columns([2, 1])라면 왼쪽이 두 배!

if option == available_options[0]:

    with col1:
        st.image(img1, width=900, caption='첫번째')

    with col2:
        st.image(img2, width=900, caption='두번째')

elif option == available_options[1]:
    with col1:
        st.image(img2, width=900, caption='첫번째')

    with col2:
        st.image(img1, width=900, caption='두번째')

else:
    st.markdown("아직 아무것도 안골랐지")

#2. 탭
tab1, tab2 = st.tabs(['Table','Graph'])

df = pd.read_csv('medical_cost.csv')
df2 = df[df['region'] == "northwest"]

with tab1:
    #df2.head(5)
    st.dataframe(df2, height=300)

with tab2:
    fig = px.scatter(data_frame=df2,
                     x='bmi', y='charges', trendline='ols',
                     width=700)
    st.plotly_chart(fig)

> 그런데 경우에 따라서는 탭 보다  <b>익스펜더(expander)</b>가 더 어울릴 수 있다. 살펴보면서 레이아웃의 어떤 점이 아쉬운지 어떻게 꾸미면 더 좋을지 계속 생각해보자!

In [ ]:
%%writefile app.py

import streamlit as st
from PIL import Image
import pandas as pd
import plotly.express as px

#페이지 설정
st.set_page_config(page_title="붕붕이", layout="wide")

#sidebar
with st.sidebar:

    st.title('사이드바')

    # selectbox
    available_options=['첫번째 이미지', '두번째 이미지']
    option = st.selectbox(label='무엇을 먼저 보고 싶니?',
                          index=None,           #아무것도 안골랐을때 할당되는 값은 None
                          options=available_options)

#본문
st.title('너가 고른 이미지')

st.header('이것이 헤더다!')

#좋지 않은 방식
imgfile1 = 'C:/Users/user/Downloads/image.jpg'
img1 = Image.open(imgfile1)
imgfile2 = 'C:/Users/user/Downloads/image2.jpg'
img2 = Image.open(imgfile2)

#1. 컬럼
col1, col2 = st.columns(2)        #st.columns([2, 1])라면 왼쪽이 두 배!

if option == available_options[0]:

    with col1:
        st.image(img1, width=900, caption='첫번째')

    with col2:
        st.image(img2, width=900, caption='두번째')

elif option == available_options[1]:
    with col1:
        st.image(img2, width=900, caption='첫번째')

    with col2:
        st.image(img1, width=900, caption='두번째')

else:
    st.markdown("아직 아무것도 안골랐지")

# 자료 불러오기 및 필터링
df = pd.read_csv('medical_cost.csv')
df2 = df[df['region'] == "northwest"]

#시각화
fig = px.scatter(data_frame=df2,
                 x='bmi', y='charges', trendline='ols',
                 width=700)
st.plotly_chart(fig)

#익스펜더
with st.expander("데이터 살펴보기"):
    st.dataframe(df)

> **연습문제 4-1**. 아래는 medical_cost.csv를 사용하여 다음 요청을 반영하는 데이터 대시보드이다. 디버깅(debugging)을 수행하시오
>
> (1) 왼쪽 사이드바에서 사용자가 성별과 연령 구간을 고를 수 있게 해주세요(e.g., `여성` 그리고 `29세에서 53세까지`).\
(2) 우측 본문은 컬럼 두 개를 나눠주세요.\
(3) 왼쪽 컬럼에 사이드에서 필터링한 데이터가 보이게 해주세요.\
(3) 우측 컬럼에 사이드에서 필터링한 데이터 안에서 `bmi`와 `charges` 간의 연관성을 시각화 해주세요.
>
> ---
> ```python
> %%writefile app.py
>
> import streamlit as st
> import pandas as pd
> import plotly.express as px
> import matplotlib.pyplot as plt
>
> #페이지 설정
> st.set_page_config(page_title="연습문제 4-1", layout="wide")
>
> # 자료 불러오기 및 필터링
> df_filtered = pd.read_csv('medical_cost.csv')
>
> # sidebar
> with st.sidebar:
>
>     st.title('데이터 필터링')
>
>     # 성별
>     gender = st.selectbox(label='분석할 성별을 고르시오',
>                           options=df['sex'].unique().tolist())   #options=['female', 'male']
>     # 나이
>     ageint = st.slider(label='분석할 연령 구간을 고르시오',
>                        min_value=df['age'].min(),
>                        max_value=df['age'].max(),
>                        value=(df['age'].min(), df['age'].max()),
>                        key='age_filter')
>
>     # 필터링
>     cond1 = (df['sex'] == gender)
>     cond2 = (df['age'] >= ageint[0]) & (df['age'] <= ageint[1])
>     df = df[cond1 & cond2]
>
>
> # 본문
> st.title('필터링된 데이터 분석')
> st.divider()
>
> data_col, viz_col = st.columns([1,2])
>
> # 데이터프레임
> with data_col:
>     st.subheader('데이터')
>     st.dataframe(df)
>
> # 시각화
> with viz_col:
>     st.subheader('시각화')
>     fig = px.scatter(data_frame=df,
>                     x='bmi', y='charges', trendline='ols')
>     st.plotly_chart(fig)
> ```

In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

#페이지 설정
st.set_page_config(page_title="연습문제 4-1", layout="wide")

# 자료 불러오기 및 필터링
df = pd.read_csv('medical_cost.csv')

# sidebar
with st.sidebar:

    st.title('데이터 필터링')

    # 성별
    gender = st.selectbox(label='분석할 성별을 고르시오',
                          options=df['sex'].unique().tolist())   #options=['female', 'male']
    # 나이
    ageint = st.slider(label='분석할 연령 구간을 고르시오',
                       min_value=df['age'].min(),
                       max_value=df['age'].max(),
                       value=(df['age'].min(), df['age'].max()),
                       key='age_filter')

    # 필터링
    cond1 = (df['sex'] == gender)
    cond2 = (df['age'] >= ageint[0]) & (df['age'] <= ageint[1])
    df_filtered = df[cond1 & cond2]


# 본문
st.title('필터링된 데이터 분석')
st.divider()

data_col, viz_col = st.columns([1,2])

# 데이터프레임
with data_col:
    st.subheader('데이터')
    st.dataframe(df_filtered)

# 시각화
with viz_col:
    st.subheader('시각화')
    fig = px.scatter(data_frame=df_filtered,
                    x='bmi', y='charges', trendline='ols')
    st.plotly_chart(fig)

## 5. 세션 상태

> 웹 어플리케이션에서 중요한 <b>세션 상태(session state)</b>를 이해하기 위해 아래 코드를 실행해보자. 예상대로 잘 작동하지 않는 것을 확인할 수 있을 것이다. 왜 그럴까? 사실 우리는 이 문제를 지난 주에 이미 예측할 수 있었다.

In [ ]:
%%writefile app.py

import streamlit as st

i = 0

# 버튼1
plus_one = st.button(label='+1')
if plus_one:
    i = i + 1

# 버튼2
minus_one = st.button(label='-1')
if minus_one:
    i = i - 1

st.text('i = ' + str(i))

> 핵심은 위젯을 조작할 때마다 스크립트(script)가 재시작한다는 것이다. 그러므로 아래와 같이 세션 상태별로 변수를 복잡하게 지정해 주어야 한다. 아래 코드 블록을 조심스럽게 살펴보자(변수 `i`를 `'i'`라고 표현하고 있는 부분도 주의하자).
```python
if 'i' not in st.session_state:
    st.session_state['i'] = 0
```

In [ ]:
%%writefile app.py

import streamlit as st

#i = 0
if 'i' not in st.session_state:
    st.session_state['i'] = 0

# 버튼1
plus_one = st.button(label='+1')
if plus_one:
    #i = i + 1
    st.session_state['i'] = st.session_state['i'] + 1

# 버튼2
minus_one = st.button(label='-1')
if minus_one:
    #i = i - 1
    st.session_state['i'] = st.session_state['i'] - 1

#st.text('i = ' + str(i))
st.text('i = ' + str(st.session_state['i']))

> **연습문제 5-1**. 위 코드를 수정하여 제대로 기능하는 CLEAR 버튼을 만드시오(이 버튼을 누르는 순간 `i`는 0이 된다).
>
> ---
> 힌트: CLEAR 버튼 안에서도 세션 상태 변수 개념을 고려하시오.

In [ ]:
%%writefile app.py

import streamlit as st

if 'i' not in st.session_state:
    st.session_state['i'] = 0

# 버튼1
plus_one = st.button(label='+1')
if plus_one:
    #i = i + 1
    st.session_state['i'] = st.session_state['i'] + 1

# 버튼2
minus_one = st.button(label='-1')
if minus_one:
    #i = i - 1
    st.session_state['i'] = st.session_state['i'] - 1

# 버튼3
clear = st.button(label='CLEAR')
if clear:
    #i = 0
    st.session_state['i'] = 0

st.text('i = ' + str(st.session_state['i']))

> **연습문제 5-2**. 아래 코드를 살펴보고 세션 상태 변수와 관련하여 왜 오류가 발생하는지 추정하시오(가능하면 오류를 직접 수정하시오).

```python
%%writefile app.py

import streamlit as st

# 세션 변수 취급된 버튼인데 왜 작동하지 않을까?
st.session_state['button_var'] = 0

plus_one = st.button(label='+1')
if plus_one:
    st.session_state['button_var'] = st.session_state['button_var'] + 1
st.text('값은 ' + str(st.session_state['button_var']))

In [ ]:
%%writefile app.py

import streamlit as st

#이제 잘 작동!
if 'button_var' not in st.session_state:            #없었을 때만 초기화
    st.session_state['button_var'] = 0

plus_one = st.button(label='+1')
if plus_one:
    st.session_state['button_var'] = st.session_state['button_var'] + 1
st.text('값은 ' + str(st.session_state['button_var']))

> 다만 <b>슬라이더(slider)</b>는 버튼과는 달리 출발점이 기억되므로 세션 상태 변수가 없어도 작동이 크게 어색하진 않다. 그래도 (예측못한 버그를 피하기 위해서라도) 세션 상태 변수를 사용할 수 있다.

In [ ]:
%%writefile app.py

import streamlit as st

# 1. 세션 상태 변수 없는 슬라이더 입력기
var = 0
var = st.slider(label='슬라이더란 이런 것이다!',
                min_value=1, max_value=100, step=5)
st.text('값은 ' + str(var))

# 2. 세션 상태 변수 취급된 슬라이더 입력기
st.slider(label='슬라이더란 이런 것이다!',
          min_value=1, max_value=100, step=5,
          key='slider_var')          # 이것!
st.text('값은 ' + str(st.session_state['slider_var']))

## 6. 완성하기

> 이제 최종적으로 지금까지 배운 모든 내용을 활용하여 데이터 대시보드를 만들어보자.
>
> 우리는 CO2_Emission.csv을 활용하여 다음의 요청을 반영한다.
>
> (1) 사용자에게 먼저 데이터 내용을 테이블로 보여주세요.\
> (2) 사용자가 분석할 자동차 클래스(`Vehicle Class`)와 엔진 크기(`Engine Size(L)`)를 왼쪽 사이드바에서 고르게 해주세요.\
> (3) 메인 페이지의 왼쪽 컬럼에선 메이커(`Make`)와 엔진 크기(`Engine Size(L)`)에 대한 상자-수염 도표를 보여주세요. \
> (4) 메인 페이지의 오른쪽 컬럼에선 엔진 크기(`Engine Size(L)`)를 $x$축으로, 사용자가 직접 고른 $y$축 간의 산점도를 그려주세요. 이때 사용자는 $y$축으로 연비 지표(`['Fuel Consumption City (L/100 km)', 'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)']`) 중 하나를 고를 수 있게 해주세요.
>
> 먼저 사용자에게 분석을 맡기기 전에 내가 직접 콜랩에서 분석해보자. 그것을 나중에 옮겨 붙이는 방식이 편리하다(Why?). 먼저 데이터를 출력하는 부분은 다음과 같다.

In [ ]:
import pandas as pd

#함수 만들기
def load_dataset(filename):
    return pd.read_csv(filename)

data = 'CO2_Emissions.csv'
df = load_dataset(data)
df.head().T      #transformed matrix

> 사용자가 골라야 하는 자동차 클래스(`Vehicle Class'`)와 엔진 크기(`Engine Size(L)`)를 천천히 살펴보자. 임의로 적당히 데이터를 필터링 해보고 그걸 시각화 해보자.

In [ ]:
import plotly.express as px

#자동차 클래스
df['Vehicle Class']
df['Vehicle Class'].unique().tolist()

#엔진 크기
df['Engine Size(L)']
df['Engine Size(L)'].describe()

# 데이터 필터링하기
cond1 = df['Vehicle Class'] == 'TWO-SEATER'
cond2 = df['Engine Size(L)'] < 5
cond3 = df['Engine Size(L)'] > 2
df.loc[cond1 & cond2 & cond3]

> 이제 첫번째 시각화를 수행해보자. 메인 페이지의 왼쪽 컬럼에선 메이커(`Make`)와 엔진 크기(`Engine Size(L)`)에 대한 상자-수염 도표를 보여준다. 이때 `data_frame=df`를 넣기보다는 적절하게 정렬(sort)하여 넣으면 그래프가 좀 더 보기 좋다.

In [ ]:
# 시각화 1
df_to_show = df.sort_values('Engine Size(L)', ascending=False)
fig1 = px.box(data_frame=df_to_show,
              x='Make', y='Engine Size(L)',
              width=800, height=400)
fig1.show()

> 두번째 시각화가 남아있다. 메인 페이지의 오른쪽 컬럼에선 엔진 크기(`Engine Size(L)`)를 $x$축으로, 사용자가 직접 고른 $y$축 간의 산점도를 그려야 한다. 사용자가 연비 지표 중 뭘 고를지는 모르지만, 일단 임의로 `Fuel Consumption Comb (L/100 km)`를 선택했다고 가정하고 시각화 해보자.

In [ ]:
# 시각화 2
chosen = 'Fuel Consumption Comb (L/100 km)'
fig2 = px.scatter(data_frame=df,
                  x='Engine Size(L)', y=chosen,
                  width=800, height=400,
                  trendline='ols')
fig2.show()

> 이제 위 코드들을 streamlit 코드에 적절히 옮겨 심는다! 먼저 사이드바부터 하나하나 해보자. 우리가 필요한 변수의 속성에 따라 적절한 입력 위젯(input widget)을 골라야 한다. 가령 자동차 클래스(`Vehicle Class`)는 범주형 변수이기 때문에 멀티셀렉트(multiselect) 정도가 무난할 것 같다. 그리고 엔진 크기(`Engine Size(L)`)는 연속형 변수이기 때문에 슬라이더(slider)가 좋을 것 같다.

In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import plotly.express as px

#페이지 설정
st.set_page_config(page_title="붕붕이와 이산화탄소 분석기", layout="wide")

# 자료 불러와 보여주기
def load_dataset(filename):
    return pd.read_csv(filename)

data = 'CO2_Emissions.csv'
df = load_dataset(data)

# 사이드바
with st.sidebar:

    #마크다운
    st.markdown('데이터 분석기 :tulip:')

    vclass = df['Vehicle Class'].unique().tolist()

    #멀티셀렉트
    st.multiselect(label = '분석하고자 하는 자동차 클래스를 모두 고르세요',
                   options = vclass,
                   default=['TWO-SEATER'],
                   key='maker_filter')                #세션 상태 변수

    #슬라이더
    st.slider(label = '분석할 엔진 크기를 설정하세요',
              min_value = df['Engine Size(L)'].min(),
              max_value = df['Engine Size(L)'].max(),
              value=(df['Engine Size(L)'].min(), df['Engine Size(L)'].max()),
              key='engine_filter')                    #세션 상태 변수

# 사용자가 고른 조건대로 데이터 필터링하기
cond1 = df['Vehicle Class'].isin(st.session_state['maker_filter'])    #maker
cond2 = df['Engine Size(L)'] >= st.session_state['engine_filter'][0]  #엔진 크기 최소
cond3 = df['Engine Size(L)'] <= st.session_state['engine_filter'][1]  #엔진 크기 최대
df_filtered = df.loc[cond1 & cond2 & cond3]

# 데이터 보여주기
st.markdown("데이터")
st.dataframe(df_filtered)

> 위 코드가 성공적으로 작동했다면 컬럼을 좌우로 나누어 설정하고, 그래프를 만든다.

In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import plotly.express as px

#페이지 설정
st.set_page_config(page_title="붕붕이와 이산화탄소 분석기", layout="wide")

# 자료 불러와 보여주기
def load_dataset(filename):
    return pd.read_csv(filename)

data = 'CO2_Emissions.csv'
df = load_dataset(data)

# 사이드바
with st.sidebar:

    #마크다운
    st.markdown('데이터 분석기 :tulip:')

    vclass = df['Vehicle Class'].unique().tolist()

    #멀티셀렉트
    st.multiselect(label = '분석하고자 하는 자동차 클래스를 모두 고르세요',
                   options = vclass,
                   default=['TWO-SEATER'],
                   key='maker_filter')                #세션 상태 변수

    #슬라이더
    st.slider(label = '분석할 엔진 크기를 설정하세요',
              min_value = df['Engine Size(L)'].min(),
              max_value = df['Engine Size(L)'].max(),
              value=(df['Engine Size(L)'].min(), df['Engine Size(L)'].max()),
              step=.3,
              key='engine_filter')                    #세션 상태 변수

# 사용자가 고른 조건대로 데이터 필터링하기
cond1 = df['Vehicle Class'].isin(st.session_state['maker_filter'])    #maker
cond2 = df['Engine Size(L)'] >= st.session_state['engine_filter'][0]  #엔진 크기 최소
cond3 = df['Engine Size(L)'] <= st.session_state['engine_filter'][1]  #엔진 크기 최대
df_filtered = df.loc[cond1 & cond2 & cond3]

# 데이터 보여주기
st.markdown("데이터")
st.dataframe(df_filtered)

# 메인 페이지: 타이틀
st.title('CO2 Emission')
st.write('웹상에서 차량의 클래스와 엔진 크기를 고르면 이산화탄소 배출량을 확인할 수 있습니다.')
st.divider()

# 컬럼 설정
col1, col2 = st.columns([3, 2])

# 메인 페이지 1: 메이커-엔진 크기 상자-수염 도표
with col1:

    st.subheader('엔진 크기')
    st.markdown('메이커 별로 엔진 크기를 상자-수염으로 나타냅니다.')

    df_to_show = df_filtered.sort_values('Engine Size(L)', ascending=False)
    fig1 = px.box(data_frame=df_to_show,
                x='Make', y='Engine Size(L)',
                width=800, height=400)
    st.plotly_chart(fig1)
    st.divider()

# 메인 페이지 2: 엔진 크기와 연비 지표 산점도
with col2:

    st.subheader('연비 분석')
    st.markdown('메이커 별로 엔진 크기를 상자-수염으로 나타냅니다.')

    st.selectbox(label = '$y$축을 고르시오',
                 options = ['Fuel Consumption City (L/100 km)', 'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'],
                 key='yaxis')

    fig2 = px.scatter(data_frame=df_filtered,
                      x='Engine Size(L)', y=st.session_state['yaxis'],
                      width=800, height=400,
                      trendline='ols')
    st.plotly_chart(fig2)
    st.divider()

> **연습문제 6-1**. 12주차 **연습문제 1-1**을 대시보드로 출력해보자. 이때 이용자가 직접 분석할 수 있도록 최대한의 인터렉티브한 환경을 제공하자. seaborn 그림과 plotly 그림을 각각 연습해보자.

In [ ]:
%%writefile app.py

import streamlit as st
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# 제목
st.header('막대 그래프를 만들어 드립니다.')

df = pd.read_csv('ds_salaries.csv')


# 사이드바
with st.sidebar:

    # groupby() 입력받기
    grpby = st.selectbox(label='$y$축을 고르시오',
                        index=3,          #none이면 아래 groupby()에서 none이 들어가므로 오류가 난다!
                        options=df.columns)

    # 평균값을 확인할 변수 입력받기
    xvar = st.selectbox(label='$x$축을 고르시오',
                        index=6,          #none이면 아래에서 none이 들어가므로 오류가 난다!
                        options=df.columns)

    # 상위 몇 명을 확인할지 입력받기
    count = st.slider(label = '상위 몇 명을 확인할지 고르시오',
                    min_value = 1, max_value = 20,
                    value = 10)          #연습문제 1-1에서 디폴트는 10

    # 센스!
    picsize  = st.sidebar.slider(label = "그림 크기",
                                 min_value = 5,
                                 max_value = 25)

# 필터링 적용
job_salary = df.groupby(grpby)[xvar].mean()
tops = job_salary.sort_values(ascending = False)[0:count]
tops = tops.reset_index()

# 시각화
fig, ax = plt.subplots(figsize=(picsize, picsize))     #물론 height과 width를 따로 받을 수 있다
sns.barplot(data=tops,
            x=xvar, y=grpby,
            ax=ax)
st.pyplot(fig)